In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("nada")
else:
    print("API Key succesfully loaded")

In [3]:
openai = OpenAI()

In [4]:
# Create a class to represent a web page

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create a website object from the given url using BeautifulSoup
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevent in soup.body(["script","style","img","input"]):
            irrelevent.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
mypage = Website("https://www.velocitydatainsights.com/")
print(mypage.title)
print(mypage.text)

In [6]:
system_prompt = """You are an AI assistance that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related.  Respond in markdown."""

In [15]:
def user_prompt_for(website):
    user_prompt = f"The website you are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
system_prompt

In [ ]:
print(user_prompt_for(mypage))

In [22]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [30]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
messages_for(mypage)

In [32]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [34]:
display_summary("https://anthropic.com")

# Summary of Anthropic Website

Anthropic is an AI safety and research company based in San Francisco, focused on creating reliable and beneficial AI systems. 

## Key Highlights
- **Claude AI Models**: 
  - **Claude 3.5 Sonnet**: The latest and most intelligent AI model, available for use.
  - **Claude 3.5 Haiku**: Also introduced alongside Sonnet.
  
- **API Access**: Users can build AI-powered applications and custom experiences with Claude.

## Recent Announcements
- **October 22, 2024**: Release of new Claude 3.5 Sonnet and Claude 3.5 Haiku models.
- **September 4, 2024**: Launch of Claude for Enterprise, catering to business applications.

## Research Focus
Anthropic emphasizes AI safety and effective alignment of AI technologies with human values, supported by their research into harmless AI feedback and constitutional AI frameworks. 

Overall, the company is committed to advancing AI technology while prioritizing safety and ethical considerations.